In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
import h5py
import joblib
from collections import Counter
from tabulate import tabulate  # For tabular display

/opt/anaconda3/envs/Dissertation/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load the cleaned data and models
df = pd.read_csv('cleaned_data_final.csv')
with h5py.File('tfidf_matrix.h5', 'r') as hdf:
    tfidf_matrix = hdf['tfidf_matrix'][:]
with h5py.File('cosine_sim_matrix.h5', 'r') as hdf:
    cosine_sim = hdf['cosine_sim_matrix'][:]
with h5py.File('svd_matrix.h5', 'r') as hdf:
    svd_matrix = hdf['svd_matrix'][:]
svd = joblib.load('svd_model.pkl')

/var/folders/c3/sr79zb3j1nx_lsk2ps2tbhl80000gn/T/ipykernel_3866/3308137746.py:2: DtypeWarning: Columns (29,37) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('cleaned_data_final.csv')


In [3]:
# Load sentiment analysis model
sentiment_analysis = pipeline("sentiment-analysis")

# Load emotion analysis model
emotion_classifier = pipeline("sentiment-analysis", model='j-hartmann/emotion-english-distilroberta-base')

# Load GPT-like model and tokenizer for extracting movie titles
model_name = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
/opt/anaconda3/envs/Dissertation/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.

In [4]:
# Function to analyze sentiment
def analyze_sentiment(query):
    """Analyze the sentiment of the user's query."""
    result = sentiment_analysis(query)[0]
    sentiment = "negative" if result['label'] == 'NEGATIVE' else "positive"
    score = result['score']
    return sentiment, score

In [5]:
# Function to analyze emotion
def analyze_emotion(query):
    """Analyze the emotion of the user's query."""
    result = emotion_classifier(query)[0]
    emotion = result['label'].lower()
    score = result['score']
    return emotion, score

In [6]:
# Function to extract movie titles using GPT-2
def extract_movie_titles(query):
    """Use GPT-2 to extract movie titles mentioned in the user's query."""
    inputs = tokenizer(query, return_tensors='pt')
    outputs = model.generate(**inputs, max_length=50, pad_token_id=tokenizer.eos_token_id)
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # Assuming the response format: "Titles: [title1, title2]"
    if "Titles:" in generated_text:
        titles = generated_text.replace("Titles:", "").strip().split(',')
        return [title.strip() for title in titles]
    else:
        return []

In [7]:
# Function to perform hybrid recommendations
def hybrid_recommendation(title, df, svd_matrix, cosine_sim_matrix, alpha=0.5, top_n=5):
    idx = df.index[df['title'].str.lower() == title.lower()].tolist()
    if idx:
        idx = idx[0]
        svd_sim = cosine_similarity(svd_matrix[idx].reshape(1, -1), svd_matrix).flatten()
        hybrid_scores = alpha * cosine_sim_matrix[idx] + (1 - alpha) * svd_sim
        top_indices = np.argsort(hybrid_scores)[-top_n-1:-1][::-1]
        return df.iloc[top_indices]
    return pd.DataFrame()

In [19]:
# Function to add diversity and serendipity to recommendations
def add_diversity_and_serendipity(recommendations, df, num_diverse=5, num_serendipitous=5):
    diverse_recs = df.sample(n=num_diverse)  # Randomly select diverse movies
    serendipitous_recs = df[df['tomatoMeter'] > 0.7].sample(n=num_serendipitous)  # Select high-rated but random movies
    combined_recs = pd.concat([recommendations, diverse_recs, serendipitous_recs])
    return combined_recs.drop_duplicates().head(5).reset_index(drop=True)

In [20]:
# Function to calculate genre diversity
def calculate_genre_diversity(recommendations):
    genres = recommendations['genre'].str.split(',').explode()
    genre_count = len(genres.unique())
    return genre_count / len(genres)

# Function to calculate similarity between recommendations
def calculate_similarity(recommendations, tfidf_matrix):
    indices = recommendations.index
    rec_matrix = tfidf_matrix[indices]
    similarity = cosine_similarity(rec_matrix)
    avg_similarity = np.mean(similarity)
    return avg_similarity

# Function to calculate intra-list similarity
def calculate_intra_list_similarity(recommendations, tfidf_matrix):
    indices = recommendations.index
    rec_matrix = tfidf_matrix[indices]
    similarity = cosine_similarity(rec_matrix)
    np.fill_diagonal(similarity, 0)
    avg_intra_similarity = np.mean(similarity)
    return avg_intra_similarity

In [21]:
# Main function to recommend movies based on sentiment and emotion
def recommend_movies(query):
    sentiment, sentiment_score = analyze_sentiment(query)
    emotion, emotion_score = analyze_emotion(query)
    titles = extract_movie_titles(query)

    recommendations = pd.DataFrame()

    # If titles are mentioned, recommend movies similar to them
    if titles:
        for title in titles:
            title_recs = hybrid_recommendation(title, df, svd_matrix, cosine_sim, alpha=0.7)
            if not title_recs.empty:
                recommendations = pd.concat([recommendations, title_recs])
                mentioned_movie = df[df['title'].str.lower() == title.lower()]
                recommendations = pd.concat([mentioned_movie, recommendations])

    # Add diversity and serendipity to the recommendations
    recommendations = add_diversity_and_serendipity(recommendations, df)

    # Evaluate the recommendations
    genre_diversity = calculate_genre_diversity(recommendations)
    similarity = calculate_similarity(recommendations, tfidf_matrix)
    intra_list_similarity = calculate_intra_list_similarity(recommendations, tfidf_matrix)

    # Print evaluation metrics
    print(f"Query Sentiment: {sentiment.capitalize()} (Score: {sentiment_score:.2f})")
    print(f"Query Emotion: {emotion.capitalize()} (Score: {emotion_score:.2f})")
    print(f"Genre Diversity: {genre_diversity:.2f}")
    print(f"Similarity: {similarity:.2f}")
    print(f"Intra-List Similarity: {intra_list_similarity:.2f}")

    # Output the recommendations in table format
    if not recommendations.empty:
        print(tabulate(recommendations[['title', 'genre']], headers='keys', tablefmt='grid'))


In [25]:
# Example usage
query = "I want Some Interesting Movies to watch"
recommended_movies = recommend_movies(query)
print(recommended_movies)

Query Sentiment: Positive (Score: 1.00)
Query Emotion: Neutral (Score: 0.75)
Genre Diversity: 0.86
Similarity: 0.12
Intra-List Similarity: 0.00
+----+----------------------+-----------------+
|    | title                | genre           |
+====+======================+=================+
|  0 | black belt jones     | adventure       |
+----+----------------------+-----------------+
|  1 | our time machine     | documentary     |
+----+----------------------+-----------------+
|  2 | officer downe        | action, fantasy |
+----+----------------------+-----------------+
|  3 | the astronaut of god | sci-fi          |
+----+----------------------+-----------------+
|  4 | bad boys             | action, comedy  |
+----+----------------------+-----------------+
None


## 